# Phase 4: Fine-tuned 모델로 인기글 생성

Phase 3에서 저장한 파인튜닝 모델을 로드하여 각 게시판별 키워드로 새로운 인기글을 생성합니다.


## ⚙️ 사전 조건
- `phase1_topic_modeling.ipynb` 실행으로 `*_topics_for_prompt.json` 파일 존재
- `phase3_finetune.ipynb` 실행으로 `models/llama3_popular_post_lora` (및 merged_16bit) 존재
- GPU 런타임 할당


---
## 0. 패키지 설치 (필요 시)
- Unsloth/transformers 가 설치되지 않았다면 Phase 3의 설치 셀을 재사용하세요.


In [ ]:
# Phase 4에서 사용하는 최소 패키지 설치
# - Unsloth: 파인튜닝된 Llama-3 모델 로드 및 추론용
# - pandas: 생성 결과를 DataFrame/CSV로 저장하기 위해 사용

%pip install -q "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git" pandas

print("✅ Phase 4 추론용 패키지 설치 완료")


In [ ]:
from google.colab import drive

# Google Drive 마운트 (이미 마운트되어 있다면 건너뛰어도 됩니다)
drive.mount('/content/drive', force_remount=True)

print("✅ Google Drive 마운트 완료")


---
## 1. 경로 설정 및 키워드 로드


In [ ]:
from pathlib import Path
import json
import pandas as pd
from IPython.display import display

PROJECT_ROOT = Path("/content/drive/MyDrive/board_crawling")
OUTPUT_DIR = PROJECT_ROOT / "outputs"
MODEL_DIR = PROJECT_ROOT / "models"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
MODEL_DIR.mkdir(parents=True, exist_ok=True)

PROMPT_FILES = {
    "익게2": OUTPUT_DIR / "익게2_topics_for_prompt.json",
    "자유게시판": OUTPUT_DIR / "자유게시판_topics_for_prompt.json",
    "연애상담소": OUTPUT_DIR / "연애상담소_topics_for_prompt.json",
    "익게1": OUTPUT_DIR / "익게1_topics_for_prompt.json",
}

trend_keywords = {}
for board, path in PROMPT_FILES.items():
    if not path.exists():
        print(f"⚠️ {board}: {path} 없음 → 건너뜀")
        continue
    with open(path, "r", encoding="utf-8") as f:
        topics_data = json.load(f)
    top_topics = sorted(topics_data, key=lambda x: len(x.get("representatives", [])), reverse=True)[:5]
    keywords = []
    for topic in top_topics:
        kw = topic.get("keywords", [])
        if isinstance(kw, list):
            keywords.extend(kw[:8])
    trend_keywords[board] = list(dict.fromkeys(keywords))[:8]
    print(f"✅ {board}: {len(trend_keywords[board])}개 키워드")

if not trend_keywords:
    raise RuntimeError("키워드가 없습니다. Phase 1 노트북을 먼저 실행하세요.")


---
## 2. 파인튜닝 모델 로드


In [ ]:
from unsloth import FastLanguageModel

FINETUNED_DIR = MODEL_DIR / "llama3_popular_post_lora"
if not FINETUNED_DIR.exists():
    raise FileNotFoundError(f"{FINETUNED_DIR}가 없습니다. Phase 3을 먼저 실행하세요.")

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=str(FINETUNED_DIR),
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=True,
)

FastLanguageModel.for_inference(model)
print("✅ 파인튜닝 모델 로드 완료")


---
## 3. 생성 함수 정의


In [ ]:
import torch

def generate_post(board_name: str, keywords: list, temperature=0.7, max_new_tokens=800):
    instruction = f"주어진 키워드를 사용하여 {board_name}의 인기글 스타일로 글을 작성해줘."
    input_text = f"키워드: {', '.join(keywords)}"
    prompt = f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input_text}

### Response:
"""
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
    )
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    if "### Response:" in generated_text:
        response = generated_text.split("### Response:")[-1].strip()
    else:
        response = generated_text.strip()
    return response


def split_title_content(text: str):
    lines = text.strip().split("\n")
    if len(lines) > 1:
        title = lines[0].strip()
        content = "\n".join(lines[1:]).strip()
    else:
        title = text[:50].strip() + "..." if len(text) > 50 else text.strip()
        content = text.strip()
    return title, content


---
## 4. 게시판별 생성 실행


In [ ]:
from datetime import datetime

generated_posts = []
for board, keywords in trend_keywords.items():
    if not keywords:
        continue
    print(f"\n생성 중: {board} ({', '.join(keywords[:5])}...)")
    try:
        generated_text = generate_post(board, keywords, temperature=0.7, max_new_tokens=800)
        title, content = split_title_content(generated_text)
        generated_posts.append({
            "board_name": board,
            "keywords": ", ".join(keywords),
            "generated_title": title,
            "generated_content": content,
            "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        })
        print(f"✅ {board} 생성 완료")
    except Exception as exc:
        print(f"⚠️ {board} 생성 실패: {exc}")


In [ ]:
if generated_posts:
    result_df = pd.DataFrame(generated_posts)
    output_csv = OUTPUT_DIR / "generated_posts.csv"
    result_df.to_csv(output_csv, index=False, encoding="utf-8-sig")
    print(f"\n✅ 생성 결과 저장 완료: {output_csv}")
    display(result_df[["board_name", "generated_title"]].head())
else:
    print("⚠️ 생성된 게시글이 없습니다.")


---
## 5. 추가 팁
- 생성 결과를 수작업으로 검토하여 품질 확인 후 배포하세요.
- 다른 샘플을 얻고 싶다면 `temperature`, `max_new_tokens`, 키워드 조합 등을 조정할 수 있습니다.
- 모델을 공유하려면 Phase 3에서 저장한 merged 모델을 업로드하거나 Hugging Face Hub에 push 하면 됩니다.
